In [ ]:
import cv2
import matplotlib.pyplot as plt
import albumentations as A
import numpy as np
import os

# --- [1] 시각화 함수 정의 (건드릴 필요 없음) ---
def visualize_augmentation(original, augmented, 
                           title1="Original (Raw)", title2="Augmented (CLAHE 2.0 + SSR)"):
    """원본과 증강된 이미지를 나란히 비교해서 보여줍니다."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # 원본 이미지 표시
    axes[0].imshow(original, cmap='gray') # 엑스레이니까 gray colormap 사용
    axes[0].set_title(title1, fontsize=15, fontweight='bold')
    axes[0].axis('off')

    # 증강 이미지 표시
    axes[1].imshow(augmented, cmap='gray')
    axes[1].set_title(title2, fontsize=15, fontweight='bold')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# --- [2] 테스트할 이미지 파일 경로 설정 (⭐ 여기를 수정하세요!) ---
# 본인 컴퓨터에 있는 실제 훈련 데이터 중 하나를 골라 경로를 입력해주세요.
# 예: '/content/data/train/DCM/image123.png' 또는 './data/train_sample.jpg'

image_path = '/data/ephemeral/home/data/train/DCM/ID301/image1664240941206.png'
# image_path = '/data/ephemeral/home/data/train/DCM/ID001/image1661130828152_R.png'

# --- [3] 이미지 로드 ---
if not os.path.exists(image_path):
    print(f"⚠️ 경고: '{image_path}' 파일을 찾을 수 없습니다!")
    print("테스트를 위해 임시로 검은 배경에 흰 사각형 이미지를 생성합니다.")
    # 테스트용 더미 이미지 생성 (경로 설정 안 했을 때용)
    image = np.zeros((1024, 1024), dtype=np.uint8)
    cv2.rectangle(image, (250, 400), (750, 900), 200, -1) # 손 모양 대충 흉내
    cv2.circle(image, (500, 300), 100, 200, -1)
else:
    # 엑스레이는 그레이스케일로 읽는 게 국룰입니다.
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# --- [4] 적용할 증강 파이프라인 정의 (시각화 전용) ---
# 🚨 중요: Normalize와 ToTensorV2는 뺍니다! 눈으로 볼 거니까요.
# 🚨 중요: p=1.0으로 설정해서 실행할 때마다 무조건 변화가 일어나게 합니다.
transform_vis = A.Compose([
    A.Resize(512, 512),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=1.0),
    A.ShiftScaleRotate(
                shift_limit=0.05,  # 상하좌우로 6.25% 정도만 살짝 이동
                scale_limit=0.05,     # 크기를 ±10% 정도 변화 (손 크기 다양화)
                rotate_limit=20,     # 회전 각도 ±20도 (너무 많이 돌리면 손가락 잘림 주의)
                p=1.0,               # 2장에 1장꼴로 적용
                border_mode=0
            ),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- [5] 증강 적용 및 결과 확인 ---
# transform 적용
transformed = transform_vis(image=image)
augmented_image = transformed['image']

# 시각화 함수 호출
print(">>> 증강 전후 비교 실행 <<<")
visualize_augmentation(image, augmented_image)

print("\n--------------------------------------------------------------")
print("💡 팁: 이 코드를 반복해서 실행(Ctrl+Enter)해보세요!")
print("   실행할 때마다 랜덤하게 회전하고 이동하는 모습을 볼 수 있습니다.")
print("   손가락 끝이 잘려 나가지 않는지 중점적으로 확인해보세요.")
print("--------------------------------------------------------------")